# 6. Классификация текстов при помощи сверточных сетей

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г. 

In [1]:
import nltk
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from nltk.corpus import stopwords, wordnet
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, random_split, DataLoader
import re
from collections import defaultdict
from functools import lru_cache

## 1. Представление и предобработка текстовых данных в виде последовательностей

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [3]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [4]:
text = text.lower()

In [5]:
words_from_text = list(set(nltk.word_tokenize(text.replace(".", "")))) #берем все слова, которые встречаются в тексте (список уникальных слов)
words_with_index = {j: i for i, j in enumerate(words_from_text)} #проставляем явный индекс словам
words_with_index

{'your': 0,
 'libtorch': 1,
 'note': 2,
 'preferences': 3,
 'and': 4,
 'available': 5,
 'is': 6,
 'run': 7,
 'the': 8,
 'command': 9,
 'select': 10,
 'c++': 11,
 'currently': 12,
 'supported': 13,
 'pytorch': 14,
 'version': 15,
 'most': 16,
 'stable': 17,
 'install': 18,
 'for': 19,
 'only': 20,
 'represents': 21,
 'that': 22,
 'tested': 23,
 'of': 24}

In [6]:
[words_with_index[word] for word in nltk.word_tokenize(nltk.sent_tokenize(text)[0].replace(".", ""))] # получаем индексы слов из словаря уникальных слов для 1 предложения

[10, 0, 3, 4, 7, 8, 18, 9]

1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [7]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [8]:
text = text.lower()

In [9]:
words_from_text = list(set(nltk.word_tokenize(text.replace(".", "")))) #берем все слова, которые встречаются в тексте (список уникальных слов)
words_with_index = {j: i for i, j in enumerate(words_from_text)} #проставляем явный индекс словам

In [10]:
words_from_first_sentence = nltk.word_tokenize(nltk.sent_tokenize(text.replace(".", ""))[0]) #берем слова из первого предложения

In [11]:
vectors = torch.zeros(len(words_from_first_sentence), len(words_from_text)) # кол-во строк = кол-во слов в предложении, кол-во столбцов = кол-во уникальных слов
indices = [(i, words_with_index[j]) for i, j in enumerate(words_from_first_sentence)]
vectors[list(zip(*indices))] = 1
vectors
# происходит one hot encoding - т.е. если слово встречается в предложении, то проставляется 1 в позиции в словаре уникальных слов

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [12]:
torch.manual_seed(0)
embeds = nn.Embedding(num_embeddings=len(words_from_text), embedding_dim=len(words_from_text)) # слой, который на вход принимает индексы слов, а на выход полезные признаки
indices = torch.tensor([words_with_index[i] for i in words_from_first_sentence]) # индексы слов из первого предложения
embeds(indices)

tensor([[ 9.0682e-01, -4.7551e-01, -8.7074e-01,  1.4474e-01,  1.9029e+00,
          3.9040e-01, -3.9373e-02, -8.0147e-01, -4.9554e-01, -3.6151e-01,
          5.8511e-01, -1.1560e+00, -1.4336e-01, -1.9474e-01, -8.5563e-02,
          1.3945e+00,  5.9690e-01, -4.8285e-01, -3.6610e-01, -1.3271e+00,
          1.6953e+00,  2.0655e+00, -2.3396e-01,  7.0732e-01,  5.8005e-01],
        [-1.1258e+00, -1.1524e+00, -2.5058e-01, -4.3388e-01,  8.4871e-01,
          6.9201e-01, -3.1601e-01, -2.1152e+00,  3.2227e-01, -1.2633e+00,
          3.4998e-01,  3.0813e-01,  1.1984e-01,  1.2377e+00,  1.1168e+00,
         -2.4728e-01, -1.3527e+00, -1.6959e+00,  5.6665e-01,  7.9351e-01,
          5.9884e-01, -1.5551e+00, -3.4136e-01,  1.8530e+00,  7.5019e-01],
        [-4.9968e-01, -1.0670e+00,  1.1149e+00, -1.4067e-01,  8.0575e-01,
         -9.3348e-02,  6.8705e-01, -8.3832e-01,  8.9182e-04,  8.4189e-01,
         -4.0003e-01,  1.0395e+00,  3.5815e-01, -2.4600e-01,  2.3025e+00,
         -1.8817e+00, -4.9727e-02, -

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса> 
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта: 
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

2.1 Считать файл surnames/surnames.csv.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
surname_df = pd.read_csv('/content/drive/MyDrive/ML_FU/Lab_6_embeddings/data/surnames.csv')
surname_df.head()

,surname,nationality
0,Woodford,English
1,Coté,French
2,Kore,English
3,Koury,Arabic
4,Lebzak,Russian


2.2 Закодировать национальности числами, начиная с 0.

In [15]:
labeler = LabelEncoder()
surname_df["target"] = labeler.fit_transform(surname_df["nationality"])
print(f"classes: {len(labeler.classes_)}")

classes: 18


In [16]:
surname_df.head(2)

,surname,nationality,target
0,Woodford,English,4
1,Coté,French,5


2.4 Реализовать класс Vocab (токен = символ)


*   добавьте в словарь специальный токен <PAD> с индексом 0
*   при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута max_seq_len

In [17]:
class Vocab:
  
  pad = "<PAD>"

  def __init__(self, series):
    unique_list = set()
    max_seq_len = 0
    #заполняем символами перечень и ищем максимальную длину
    for i in map(str.lower, series):
      unique_list.update(i)
      max_seq_len = max(len(i), max_seq_len)

    self.symbols = [self.pad, *unique_list] #в начало проставляем pad
    self.max_seq_len = max_seq_len #максимальная длина
    self.symb_ind = {ch: i for i, ch in enumerate(self.symbols)} #сивол - индекс

  #кодирование последовательности
  def encode(self, word):
    word = word.lower() #если вдруг на вход поступит последовательность с заглавными буквами
    indices = [self.symb_ind[symb] for symb in word]
    # дополняем индексом служебного символа
    indices += [self.symb_ind[self.pad]] * (self.max_seq_len - len(indices))
    return torch.tensor(indices, dtype=torch.long)

  #расшифровка закодированной последовательности
  def decode(self, indices):
    indices_2 = torch.nonzero(indices == self.symb_ind[self.pad], as_tuple=True)[0]
    if len(indices_2):
      indices = indices[:indices_2[0]]  #без служебных символов
    return "".join(self.symbols[i] for i in indices)

In [18]:
vocab = Vocab(surname_df["surname"])
encode = vocab.encode("Khamikoeva")
print(encode, vocab.decode(encode))

tensor([32, 46, 39, 35,  9, 32, 25, 51, 20, 39,  0,  0,  0,  0,  0,  0,  0]) khamikoeva


2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса> 
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины


In [19]:
class SurnamesDataset(Dataset):
  def __init__(self, data, vocab, transform):
    self.surname = data["surname"].tolist()
    if transform:
      size = transform(self.surname[0]).size() # сохраняем размер элемента датасета
      self.data = torch.vstack([transform(i) for i in self.surname]).view(len(self.surname), *size) # кодируем каждую фамилию, vstack - вдоль вертикали объединяем тензор, view - восстанавливаем размер
    else:
      self.data = self.surname
    self.targets = torch.tensor(data["target"], dtype=torch.long)

    self.vocab = vocab
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, i):
    return self.data[i], self.targets[i]

In [20]:
def to_index(word):
  return vocab.encode(word) #кодируем

def one_hot(word):
  vectors = torch.zeros(vocab.max_seq_len, len(vocab.symbols))
  indices = [(i, vocab.symb_ind[symb]) for i, symb in enumerate(word.lower())]
  vectors[list(zip(*indices))] = 1
  return vectors

In [21]:
#для задания 2.6
# датасет для обучения без Embedding (слова закодированы one-hot методом)
surname_one_hot = SurnamesDataset(surname_df, vocab, transform=one_hot)
# датасет для обучения через Embedding
surname_indices = SurnamesDataset(surname_df, vocab, transform=to_index)
surname_one_hot[0], surname_indices[0]

((tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 

2.3 Разбить датасет на обучающую и тестовую выборку

In [22]:
def split_train_test(data, train_part): #делим на обучающую и тестовую выборки
  train_size = round(train_part * len(data)) #размер обучающей
  test_size = len(data) - train_size #размер тестовой
  train, test = random_split(data, lengths=(train_size, test_size)) #делим
  return train, test

In [23]:
torch.manual_seed(0)

train_indices, test_indices = split_train_test(surname_indices, train_part=0.8)
train_one_hot, test_one_hot = split_train_test(surname_one_hot, train_part=0.8)
print(f'Train size: {len(train_indices)}, Test size: {len(test_indices)}')

Train size: 8784, Test size: 2196


2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта: 
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

In [24]:
class SurnamesClassifier(nn.Module):
  def __init__(self, vocab: Vocab, out_features: int, embed_dim: int=128, use_embedding: bool=True, debug: bool=False):
    
    super(SurnamesClassifier, self).__init__()
        
    self.use_embedding = use_embedding
    self.debug = debug
    self.embed_dim = embed_dim

    last_out_channels = 64 
    adaptive_avg_pool = 8

    self.embedding = nn.Embedding(num_embeddings=len(vocab.symbols), embedding_dim=embed_dim) # слой, который дает признаки из индексов слов
    self.features = nn.Sequential(
      nn.Conv1d(in_channels=embed_dim, out_channels=64, kernel_size=3), # сверточный слой, embed_dim - кол-во признаков от эмбеддинга, out_channels - выходные слои, kernel_size - вход уменьшится на 2 с каждой стороны
      nn.BatchNorm1d(num_features=64), # иногда улучшает точность
      nn.ReLU(), # функция активации
      nn.MaxPool1d(kernel_size=2), # уменьшает размерность в 2 раза
      nn.Conv1d(in_channels=64, out_channels=last_out_channels, kernel_size=3), # сверточный слой, кол-во каналов = кол-ву каналов из предыдущего сверточного слоя
      nn.BatchNorm1d(num_features=last_out_channels), # повышение точности
      nn.ReLU(), # функция активации
      nn.MaxPool1d(kernel_size=2), # уменьшает размерность еще в 2 раза
    )
    self.avgpool = nn.AdaptiveAvgPool1d(adaptive_avg_pool) # слой для контроля размерности - указываем необходимую размерность
    self.classifier = nn.Sequential(
      nn.Linear(last_out_channels * adaptive_avg_pool, 256), # линейный слой, где размерность на вход = 64*8
      nn.ReLU(),
      nn.Dropout(),
      nn.Linear(256, out_features), # Линейный слой
    )

    if self.debug:
      self.forward = self._debug_forward
    else:
      self.forward = self._forward

  def _forward(self, x):
    if self.use_embedding:
      x = self.embedding(x)
    else:
      x = F.pad(x, (0, self.embed_dim - x.size(2), 0, 0), value=0)

    x = x.reshape(x.size(0), x.size(2), x.size(1)) # ставит на место размерности (последняя и предпоследняя меняются местами)
    x = self.features(x) # свертка
    x = self.avgpool(x) # меняем размерность
    x = torch.flatten(x, 1) # нужно привести к количеству измерений = 2
    x = self.classifier(x) # линейные слои
    return torch.log_softmax(x, dim=1) # наблюдение за ошибкой

  def _debug_forward(self, x):
    print("x: ", x.size())
    if self.use_embedding:
      x = self.embedding(x)
      print("embedding: ", x.size())
    else:
      x = F.pad(x, (0, self.embed_dim - x.size(2), 0, 0), value=0)
      print("pad: ", x.size())

    x = x.reshape(x.size(0), x.size(2), x.size(1))
    print("reshape: ", x.size())
    x = self.features(x)
    print("features: ", x.size())
    x = self.avgpool(x)
    print("avgpool: ", x.size())
    x = torch.flatten(x, 1)
    print("flatten: ", x.size())
    x = self.classifier(x)
    print("classifier: ", x.size())
    return torch.log_softmax(x, dim=1)

In [25]:
if torch.cuda.is_available():
  DEVICE = "cuda"
else:
  DEVICE = "cpu"

In [26]:
# def on_cuda(device):
#   return device == "cuda"

def train_func(dataloader, model, loss_fn, optimizer, verbose, device):
  
  model.train()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  avg_loss = 0

  for batch, (x, y) in enumerate(dataloader):
    x, y = x.to(device), y.to(device)

    pred = model(x)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    avg_loss += loss
    if batch % verbose == 0:
      print(f'loss: {loss:>7f}  [{batch * len(x):>5d} / {size:>5d}]')

    del x, y, pred, loss
    torch.cuda.empty_cache()

  return avg_loss / num_batches


@torch.no_grad() # не считать градиенты для экономии памяти и времени
def test_func(dataloader, model, loss_fn, device):
  model.eval()

  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  avg_loss, correct = 0, 0

  for x, y in dataloader:
    x, y = x.to(device), y.to(device)
    pred = model(x)
    avg_loss += loss_fn(pred, y)
    correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    del x, y, pred
    torch.cuda.empty_cache()

  avg_loss /= num_batches
  accuracy = correct / size
  print(f'Accuracy: {accuracy}, Avg loss: {avg_loss} \n')

  return avg_loss, accuracy

def common(model, loss_fn, optimizer, train_dataloader, epochs, test_dataloader, lr_scheduler=None, verbose: int=100, device: str='cpu'):
    
  train_losses = []
  for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n" + "_" * 40)
    train_loss = train_func(train_dataloader, model, loss_fn, optimizer, verbose=verbose, device=device)
    train_losses.append(train_loss.item())
    if test_dataloader:
      loss, acc = test_func(test_dataloader, model, loss_fn, device=device)
      if lr_scheduler:
        lr_scheduler.step(loss)
    torch.cuda.empty_cache()
  return train_losses

In [27]:
torch.manual_seed(0)

common_net = SurnamesClassifier(vocab, len(labeler.classes_)).to(DEVICE)
loss_fn = nn.NLLLoss()
optimizer = optim.Adam(common_net.parameters(), lr=0.001)

In [28]:
%%time

# One-Hot
common_net.use_embedding = False
hot = common(
    epochs=10,
    model=common_net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=DataLoader(train_one_hot, batch_size=8, shuffle=True),
    test_dataloader=DataLoader(test_one_hot, batch_size=512),
    verbose=500,
    device=DEVICE
)

Epoch 1
________________________________________
loss: 2.915266  [    0 /  8784]
loss: 1.868670  [ 4000 /  8784]
loss: 1.461547  [ 8000 /  8784]
Accuracy: 0.5368852459016393, Avg loss: 1.6396381855010986 

Epoch 2
________________________________________
loss: 1.670395  [    0 /  8784]
loss: 1.822924  [ 4000 /  8784]
loss: 0.838009  [ 8000 /  8784]
Accuracy: 0.5897085610200364, Avg loss: 1.443270206451416 

Epoch 3
________________________________________
loss: 0.759084  [    0 /  8784]
loss: 1.151868  [ 4000 /  8784]
loss: 2.312820  [ 8000 /  8784]
Accuracy: 0.6020036429872495, Avg loss: 1.3618360757827759 

Epoch 4
________________________________________
loss: 1.303294  [    0 /  8784]
loss: 1.555584  [ 4000 /  8784]
loss: 1.680857  [ 8000 /  8784]
Accuracy: 0.6183970856102003, Avg loss: 1.3246428966522217 

Epoch 5
________________________________________
loss: 1.955585  [    0 /  8784]
loss: 1.066343  [ 4000 /  8784]
loss: 0.126044  [ 8000 /  8784]
Accuracy: 0.6238615664845173, Av

Можно увидеть, что точность в среднем равняется 62%. Это не такие хорошие результаты.

In [29]:
torch.manual_seed(0)

In [30]:
torch.manual_seed(0)
embeddings_net = SurnamesClassifier(vocab, len(labeler.classes_)).to(DEVICE)
loss_fn = nn.NLLLoss()
optimizer = optim.Adam(embeddings_net.parameters(), lr=0.001)

In [31]:
%%time

# Embedding
embeddings_net.use_embedding = True
emb = common(
    epochs=15,
    model=embeddings_net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=DataLoader(train_indices, batch_size=8, shuffle=True),
    test_dataloader=DataLoader(test_indices, batch_size=512),
    verbose=500,
    device=DEVICE,
)

Epoch 1
________________________________________
loss: 2.909279  [    0 /  8784]
loss: 1.590480  [ 4000 /  8784]
loss: 1.432680  [ 8000 /  8784]
Accuracy: 0.5491803278688525, Avg loss: 1.624413251876831 

Epoch 2
________________________________________
loss: 1.999613  [    0 /  8784]
loss: 1.635629  [ 4000 /  8784]
loss: 1.539364  [ 8000 /  8784]
Accuracy: 0.6388888888888888, Avg loss: 1.331519603729248 

Epoch 3
________________________________________
loss: 1.428237  [    0 /  8784]
loss: 1.474102  [ 4000 /  8784]
loss: 2.516058  [ 8000 /  8784]
Accuracy: 0.6425318761384335, Avg loss: 1.2822692394256592 

Epoch 4
________________________________________
loss: 1.341563  [    0 /  8784]
loss: 0.821681  [ 4000 /  8784]
loss: 0.481285  [ 8000 /  8784]
Accuracy: 0.6461748633879781, Avg loss: 1.2427399158477783 

Epoch 5
________________________________________
loss: 1.201291  [    0 /  8784]
loss: 1.654966  [ 4000 /  8784]
loss: 0.556950  [ 8000 /  8784]
Accuracy: 0.6712204007285975, Avg

Можно заметить, что точность в среднем 70%, что выше, чем в предыдущем эксперименте. Однако скорость выполнения ниже.

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [32]:
test_func(dataloader=DataLoader(test_indices, batch_size=512), model=embeddings_net, loss_fn=loss_fn, device=DEVICE);

Accuracy: 0.7144808743169399, Avg loss: 1.240553617477417 



Измерим точность нашей модели на перечне фамилий студентов в качестве данных.

In [33]:
def total_conclusion(surname, target, model, vocab, labeler, n, device):
  x = vocab.encode(surname.lower())
  x = x.to(device)

  pred = model(x.unsqueeze(0))
  pred_prob, pred_label_indices = F.softmax(pred, 1).topk(n, dim=1)
  pred_labels = labeler.inverse_transform(pred_label_indices.squeeze().cpu())

  predict = ", ".join([f"{label} ({prob})" for (label, prob) in zip(pred_labels, pred_prob.squeeze())])
  print(f'Surname.....{surname}')
  print(f'True........{target}')
  print(f'Predicts....{predict}\n')

In [34]:
students_PI19_3 = [
  "Alexandrova",
  "Baranov",
  "Brusova",
  "Volkova",
  "Gasanova",
  "Danilin",
  "Demenchuk",
  "Egorov",
  "Popova",
  "Polikarpova",
  "Khamikoeva",
]

for surname in students_PI19_3:
  total_conclusion(surname=surname, target="Russian", model=embeddings_net, vocab=vocab, labeler=labeler, n=3, device=DEVICE)

Surname.....Alexandrova
True........Russian
Predicts....Russian (0.9935316443443298), English (0.004402637481689453), French (0.00096506456611678)

Surname.....Baranov
True........Russian
Predicts....Russian (1.0), Czech (1.0124450648511807e-13), French (5.999957204249479e-14)

Surname.....Brusova
True........Russian
Predicts....Russian (0.7696868777275085), Czech (0.13018909096717834), Japanese (0.03549609333276749)

Surname.....Volkova
True........Russian
Predicts....Russian (0.999427855014801), Czech (0.0005587751511484385), Polish (6.436499916162575e-06)

Surname.....Gasanova
True........Russian
Predicts....Russian (0.999630331993103), Japanese (0.00017946900334209204), Czech (8.325811359100044e-05)

Surname.....Danilin
True........Russian
Predicts....Russian (0.9999924898147583), English (5.488362603500718e-06), French (1.6447306734335143e-06)

Surname.....Demenchuk
True........Russian
Predicts....Russian (0.5304750204086304), German (0.20539936423301697), English (0.1359450668096

Можно заметить, что использование embedding увеличивает точность модели. Точность предсказания очень высокая при классификации практически всех фамилий.

## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding` 
    - подберите адекватную размерность вектора эмбеддинга: 
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста


In [35]:
STOPWORDS = set(stopwords.words("english"))

In [36]:
def get_pos(word): # части речи wordnet и nltk по-разному называются, тут переводится к частям речи wordnet
  tag = nltk.pos_tag([word])[0][1]
  if tag.startswith('J'):
    return wordnet.ADJ
  elif tag.startswith('V'):
    return wordnet.VERB
  elif tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

#предобработка
def preprocess_review(text):
  #приводим к нижнему регистру
  text = text.lower()
  #регулярным выражением удаляем все символы кроме букв латинского алфавита
  text = re.sub(r"[^a-z]", repl=" ", string=text, flags=re.MULTILINE)

  lemmatizer = nltk.WordNetLemmatizer()
  words = []
  for word in nltk.word_tokenize(text):
    if word not in STOPWORDS:  # отсеиваем стоп-слова перед лемматизацией, чтобы лишнее не приводить
      lemma = lemmatizer.lemmatize(word, pos=get_pos(word))
      # избавляемся еще от части слов
      if lemma not in STOPWORDS and len(lemma) > 2:
        words.append(lemma)

  return " ".join(words)

In [37]:
#создаем класс для формирования набора данных
class ReviewsDataset(Dataset):

  def __init__(self, positive, negative, seed):
    self.positive = positive #пути к данным
    self.negative = negative
    self.positive_reviews = self.read_reviews(positive, preprocess_review) #читает и обрабатывает данные
    self.negative_reviews = self.read_reviews(negative, preprocess_review)

    data = self.positive_reviews + self.negative_reviews # слияние данных
    targets = torch.cat([torch.ones(len(self.positive_reviews)), torch.zeros(len(self.negative_reviews))]) #слияние данных для классов

    if seed is not None:
      torch.manual_seed(seed)
    indices = torch.randperm(len(data)) # генераци индексов в случайном порядке для того, чтобы перемешать данные

    self.data = [data[i] for i in indices] # перемешиваются данные 
    self.targets = targets[indices].to(torch.long) # перемешиваются targets

  @staticmethod
  def read_reviews(path, process): # для обработки данных перед слиянием
    reviews = []
    with open(path) as f:
      for review in f.readlines():
        review = process(review)
        if review:
          reviews.append(review)
    return reviews

  def __len__(self):
    return len(self.data)

  def __getitem__(self, i):
    return self.data[i], self.targets[i]

In [38]:
reviews_dataset = ReviewsDataset(
  '/content/drive/MyDrive/ML_FU/Lab_6_embeddings/data/positive_reviews.txt',
  '/content/drive/MyDrive/ML_FU/Lab_6_embeddings/data/negative_reviews.txt',
    seed=0
)
len(reviews_dataset), reviews_dataset[0]

(10660, ('play less like come age romance infomercial', tensor(0)))

In [39]:
torch.manual_seed(0)

#разделим данные на обучающую и тестовую выборки
train_reviews, test_reviews = split_train_test(reviews_dataset, train_part=0.8)
len(train_reviews), len(test_reviews)

(8528, 2132)

In [40]:
class ReviewsVocab:
  pad = "<PAD>"
  unknown = "<UNK>"

  def __init__(self, reviews):
    unique_list = set()
    max_seq_len = 0
    for review in reviews:
      words = nltk.word_tokenize(review)
      unique_list.update(words)
      max_seq_len = max(len(words), max_seq_len)

    self.symbols = [self.pad, self.unknown, *unique_list]
    self.max_seq_len = max_seq_len

    word_ind = {word: i for i, word in enumerate(self.symbols)}
    # 1 - индекс служебного символа для отсутствующего ключа
    self.word_ind = defaultdict(lambda: 1, word_ind)

  def __len__(self):
    return len(self.symbols)

  #Как и в прошлый раз функции кодирования и декодирования
  
  @lru_cache(maxsize=8192) 
  def encode(self, review):
    indices = [self.word_ind[word] for word in nltk.word_tokenize(review)]
    indices += [self.word_ind[self.pad]] * (self.max_seq_len - len(indices))
    return torch.tensor(indices, dtype=torch.long)

  def decode(self, indices):
    pad_indices = torch.nonzero(indices == self.word_ind[self.pad], as_tuple=True)[0]
    if len(pad_indices):
      indices = indices[:pad_indices[0]] #без служебных символов
    return " ".join(self.symbols[i] for i in indices)

In [41]:
vocab = ReviewsVocab([review for review, _ in train_reviews])
print(f'Symbols: {len(vocab)}, Max len: {vocab.max_seq_len}')
encoded = vocab.encode('this is shy review')
encoded, vocab.decode(encoded)

Symbols: 13229, Max len: 29


(tensor([   1,    1, 9836,  424,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]), '<UNK> <UNK> shy review')

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding` 
    - подберите адекватную размерность вектора эмбеддинга: 
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

In [42]:
class ReviewsClassifier(nn.Module):
  last_out_channals = 64
  adaptive_avg_pool = 8

  def __init__(self, num_embeddings, embedding_dim):
    super(ReviewsClassifier, self).__init__()

    #строим модель
    self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim)
    self.features = nn.Sequential(
      nn.Conv1d(in_channels=embedding_dim, out_channels=self.last_out_channals, kernel_size=2), # сверточный слой, embedding_dim - задается любая размерность
      nn.BatchNorm1d(num_features=self.last_out_channals), # для улучшения работы модели
      nn.ReLU(), # функция активации
      nn.MaxPool1d(kernel_size=2), # уменьшение размерности в 2 раза
    )
    self.avgpool = nn.AdaptiveAvgPool1d(self.adaptive_avg_pool) # приведение к нужной размерности
    self.classifier = nn.Sequential(
      nn.Linear(self.last_out_channals * self.adaptive_avg_pool, 256),
      nn.ReLU(),
      nn.Dropout(),
      nn.Linear(256, 2),
    )

  def forward(self, x: torch.Tensor):
    x = self.embedding(x) # приводим данные к значениям эмбеддинга (индексы слов в признаки)
    x = x.reshape(x.size(0), x.size(2), x.size(1)) # меняем последнюю и предпоследнюю размерности
    x = self.features(x) # сверточный слой
    x = self.avgpool(x) # приведение к нужной размерности
    x = torch.flatten(x, 1) # размерность, без которой не сработает линейный слой
    x = self.classifier(x) # линейный слой
    return x


def collate(batch): # функция между даталоадером и формированием батча для кодирования данных в батче
  x_s, y_s = [], []
  for x, y in batch:
    x_s.append(vocab.encode(x))
    y_s.append(y)
  return torch.vstack(x_s), torch.hstack(y_s)

In [43]:
torch.manual_seed(0)

net = ReviewsClassifier(num_embeddings=len(vocab), embedding_dim=128).to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00091)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau( # для формирование lr
    optimizer=optimizer,
    mode="min", # стремимся к минимизации параметра
    patience=5, # каждые 5 эпох
    factor=0.33, # насколько умножается изначальный lr по прошествии 5 эпох, если ошибка не снизилась больше чем на threshold
    min_lr=0.000001,
    threshold=0.001,
    verbose=True,
)

In [44]:
train_dataloader = DataLoader(train_reviews, batch_size=22, collate_fn=collate, shuffle=True) # shuffle - на каждую новую эпоху данные в разном порядке
test_dataloader = DataLoader(test_reviews, batch_size=512, collate_fn=collate)

In [45]:
%%time

_ = common(
    epochs=20,
    model=net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    lr_scheduler=lr_scheduler,
    verbose=150,
    device=DEVICE,
)

Epoch 1
________________________________________
loss: 0.659027  [    0 /  8528]
loss: 0.713437  [ 3300 /  8528]
loss: 0.700269  [ 6600 /  8528]
Accuracy: 0.4910881801125704, Avg loss: 0.6932711005210876 

Epoch 2
________________________________________
loss: 0.690713  [    0 /  8528]
loss: 0.697010  [ 3300 /  8528]
loss: 0.699278  [ 6600 /  8528]
Accuracy: 0.4915572232645403, Avg loss: 0.6936709880828857 

Epoch 3
________________________________________
loss: 0.688256  [    0 /  8528]
loss: 0.690769  [ 3300 /  8528]
loss: 0.681394  [ 6600 /  8528]
Accuracy: 0.49906191369606, Avg loss: 0.6930369138717651 

Epoch 4
________________________________________
loss: 0.698211  [    0 /  8528]
loss: 0.729387  [ 3300 /  8528]
loss: 0.703668  [ 6600 /  8528]
Accuracy: 0.550187617260788, Avg loss: 0.6871483325958252 

Epoch 5
________________________________________
loss: 0.658416  [    0 /  8528]
loss: 0.655683  [ 3300 /  8528]
loss: 0.612282  [ 6600 /  8528]
Accuracy: 0.6027204502814258, Avg 

batch_size важный параметр, потому что если неверно его подобрать, мы застреваем в неудачном состоянии модели.

С увеличением эпохи тествая ошибка начинает очень сильно расти. Это явная проблема модели.

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%

In [46]:
@torch.no_grad() # не считать градиенты 
def get_y_test_y_pred(model, test_dataloader, device):
    
  model.eval()
  y_test = []
  y_pred = []
  for x, y in test_dataloader:
    x, y = x.to(device), y.to(device)
    pred = model(x).argmax(1)
    y_test.append(y)
    y_pred.append(pred)

    del x
    torch.cuda.empty_cache()

  return torch.hstack(y_test).detach().cpu(), torch.hstack(y_pred).detach().cpu()

In [47]:
y_test, y_pred = get_y_test_y_pred(net, test_dataloader, DEVICE)

print(metrics.classification_report(
  y_true=y_test,
  y_pred=y_pred,
  target_names=["negative", "positive"],
))

              precision    recall  f1-score   support

    negative       0.68      0.67      0.67      1061
    positive       0.68      0.69      0.69      1071

    accuracy                           0.68      2132
   macro avg       0.68      0.68      0.68      2132
weighted avg       0.68      0.68      0.68      2132



Наблюдаем результаты работы модели. 

Все метрики показывают +-68%.
Таким образом по отчету метрик классификации можно сделать вывод, что особой несбалансированности между метриками нет, поэтому accuracy тоже можно верить.
Точность = 68%.


In [48]:
def total_conclusion(review, target, model, vocab, target_names, device):
  x = vocab.encode(preprocess_review(review))
  x = x.to(device)
  pred = model(x.unsqueeze(0))
  pred_prob, pred_label_idx = F.softmax(pred, 1).max(dim=1)
  pred_label = target_names[pred_label_idx.cpu()]

  print(f'Review : {review}')
  print(f'True   : {target}')
  print(f'Predict: {pred_label} ({pred_prob.item()})\n')

In [49]:
reviews = [
  ('A terrible plot. I never seen anything so bad.', 'negative'),
  ('A fascinating story. I think the music is just beautiful.', 'positive'),
]
for review, target in reviews:
  total_conclusion(review=review, target=target, model=net, vocab=vocab, target_names=["negative", "positive"], device=DEVICE)

Review : A terrible plot. I never seen anything so bad.
True   : negative
Predict: negative (0.9999692440032959)

Review : A fascinating story. I think the music is just beautiful.
True   : positive
Predict: positive (0.999440610408783)



Модель показывает прекрасные результаты.

**Как итог:**

Батч -  важный параметр, от которого зависит, в каком оптимуме будет находиться модель. (если не то выбрать, то вообще может точность полететь)

Чем больше слов мы убираем из данных, тем меньше шума у нас получается. Сюда относятся и стопслова, и слишком короткая лемма.